# Pobranie danych oraz wybór dla wskazanych klas do dalszej analizy


---



In [14]:
import pickle
import glob
import numpy as np
import pandas as pd

In [15]:
def unpickle(file):
    all_dict = []
    meta_dict = {}
    for f in glob.glob(file):
      if f.endswith('.meta')!=True:
        print(f)
        with open(f, 'rb') as fo:
          dict = pickle.load(fo, encoding='bytes')
        all_dict.append(dict)
      else:
          with open(f, 'rb') as fo:
           meta_dict = pickle.load(fo, encoding='bytes')
    return all_dict, meta_dict

def extraction_data(data,index_my_categories):
  pic_data = data[b"data"]
  labels = data[b"labels"]
  pictures =  np.concatenate([pic_data.loc[i] for i in range(0,pic_data.shape[0])],axis=0)
  index =  np.concatenate([labels.loc[i] for i in range(0,labels.shape[0]) ],axis=0)
  return extraction_categories(index_my_categories, index, pictures)

def extraction_categories(index, index_all, pic_data):
   data = np.array([ pic_data[i] for i in range(len(index_all)) if index_all[i] in index ])
   label = np.array([ i for i in index_all if i in index ])
   return data, label

def extraction_meta(meta_d,key,type_img):
  data = meta[key]
  data = [i.decode("utf-8") for i in data]
  index_list = [data.index(i) for i in type_img]
  return index_list


In [16]:
file = "/data/Train/*"
data, meta = unpickle(file)

/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_3
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_2
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_5
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_1
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_4


In [17]:
index = extraction_meta(meta,b'label_names',["airplane", "cat", "truck"])
X_train_data,y_train_data = extraction_data(pd.DataFrame(data),index)

#Przetwarzanie wstępne


---



In [18]:
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [19]:
def preproces(pictures):
    pictures_copy = pictures.copy()
    scaler =  MinMaxScaler(feature_range=(0, 1))
    stand_pic = scaler.fit_transform(pictures_copy)
    return stand_pic

In [20]:
x_train_pre = preproces(X_train_data)
pca = PCA(n_components=40) 
pca = pca.fit(x_train_pre)
X_train = pca.transform(x_train_pre)

#Klasyfikacja danych


---




In [21]:
from sklearn.svm import SVC

In [22]:
model = SVC(C=4, gamma='scale', kernel='rbf',decision_function_shape = 'ovo',shrinking=False)
clf = model.fit(X_train, y_train_data)

#Walidacja modelu (dla danych testowych) wraz z wyznaczeniem i wydrukiem macierzy pomyłek oraz wartości F1


---



In [23]:
from sklearn.metrics import f1_score, classification_report,confusion_matrix
from sklearn import metrics

In [24]:
file = "/data/Test/*"
data, meta = unpickle(file)
X_test_data,y_test = extraction_data(pd.DataFrame(data),index)

X_test_pre= preproces(X_test_data)

X_test = pca.transform(X_test_pre)

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred),"\n")
print("Confusion matrix",confusion_matrix(y_test, y_pred, labels=[0,3,9]))
print("\nF1-score",f1_score(y_test, y_pred, average='weighted'))
print("Raport klasyfikacji",classification_report(y_test, y_pred, target_names=["0","3","9"]))

/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/test/test_batch
Accuracy: 0.82 

Confusion matrix [[831  82  87]
 [ 72 838  90]
 [ 99 110 791]]

F1-score 0.8198824607781715
Raport klasyfikacji               precision    recall  f1-score   support

           0       0.83      0.83      0.83      1000
           3       0.81      0.84      0.83      1000
           9       0.82      0.79      0.80      1000

    accuracy                           0.82      3000
   macro avg       0.82      0.82      0.82      3000
weighted avg       0.82      0.82      0.82      3000

